<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/15_optimization/040_Global_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
import numpy as np
import scipy.optimize as so
import matplotlib.pyplot as plt



# Global Optimization Examples<br>전역최적화 사례



Let's think about a cost function with multiple local minima.<br>여러 국소 최소점을 가진 비용 함수를 생각해 보자.



In [ ]:
def a_func_not_smooth(x):
    return np.sin(x*10) + 0.25 * x * x



In [ ]:
def df_dx_cost(x):
    return 10.0 * np.cos(x * 10.0) + 0.5 * x



In [ ]:
def plot_the_func_not_smooth():
    x_array = np.linspace(-10, 10, 200)
    y_array = a_func_not_smooth(x_array)
    plt.plot(x_array, y_array)
    plt.grid(True)


plot_the_func_not_smooth()
plt.show()
plt.close()



## A case of a Local Optimization Algorithm<br>국소 최적화 알고리듬의 경우



A local optimization might have challenges finding the global minimum.<br>국소 최적화 알고리듬은 "충분히 가깝지 않은" 경우 전역 최적점을 찾기 힘들 수도 있다.



In [ ]:
not_the_best_initial_guess = -0.7
result_not_the_best_initial_guess = so.minimize(
    a_func_not_smooth, not_the_best_initial_guess,
    jac=df_dx_cost,
)

plot_the_func_not_smooth()
plt.plot(
  not_the_best_initial_guess,
    a_func_not_smooth(not_the_best_initial_guess),
  'x', label="initial"
)

plt.plot(
  result_not_the_best_initial_guess.x,
    a_func_not_smooth(result_not_the_best_initial_guess.x),
  'o', label="final"
)

plt.legend(loc=0)
plt.show()
plt.close()



## Cases of Global Optimization Algorithms<br>전역 최적화 알고리듬의 경우



Declare another cost function that will plot intermediate results<br>중간 과정의 그래프를 그려 주는 비용 함수를 선언



In [ ]:
class RoughCostFunctionLogger():
    def __init__(self):

        self.x_log = []
        self.y_log = []

        self.x_plot = np.linspace(-10, 10, 201)
        self.y_plot = a_func_not_smooth(self.x_plot)

    def plot_cost_function(self):
        plt.plot(self.x_plot, self.y_plot)
        plt.grid(True)

    def cost(self, x:np.ndarray) -> float:
        result = a_func_not_smooth(x)

        self.x_log.append(x[0])
        self.y_log.append(result)

        return result

    def plot_cost_function_and_log(self):
        fig, ax = plt.subplots(figsize=(16, 9))
        self.plot_cost_function()
        plt.scatter(self.x_log, self.y_log, c=range(len(self.x_log)))
        plt.colorbar()



### Basin Hopping



Uses Monte Carlo method<br>
몬테카를로법을 적용 (무작위 시도)



In [ ]:
logger_basinhopping = RoughCostFunctionLogger()



In [ ]:
%%time
result_basinhopping = so.basinhopping(logger_basinhopping.cost, [-0.7])
result_basinhopping



In [ ]:
logger_basinhopping.plot_cost_function_and_log()
plt.axvline(result_basinhopping.x, color="red");
plt.show()
plt.close()



### Brute



Evaluate the cost function at the grid points within the given range<br>
주어진 범위 안에 매개변수 격자를 만들고, 각 격자점에서 비용함수를 계산.



In [ ]:
logger_brute = RoughCostFunctionLogger()



In [ ]:
%%time
result_brute = so.brute(logger_brute.cost, ((-5.0, 5.0),))
result_brute, a_func_not_smooth(result_brute)



In [ ]:
logger_brute.plot_cost_function_and_log()
plt.axvline(result_brute, color="red");
plt.show()
plt.close()



### Differential Evolution



Evolution of population : crossover and mutation<br>
유전자 모음을 만들고 교배와 변이를 통해 탐색



In [ ]:
logger_de = RoughCostFunctionLogger()



In [ ]:
%%time
result_de = so.differential_evolution(logger_de.cost, ((-6.0, 6.0),))
result_de



In [ ]:
logger_de.plot_cost_function_and_log()
plt.axvline(result_de.x, color="red");
plt.show()
plt.close()



### SHGO



LHS vs grid 2D example<br>
라틴 하이퍼큐브와 단순 격자 2D 비교<br>
ref :<br>
* AskUp & https://chat.openai.com
* https://en.wikipedia.org/wiki/Latin_hypercube_sampling



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc

# Number of samples per dimension
n = 10

# Generate Latin Hypercube Sampling
lhs_engine = qmc.LatinHypercube(d=2, seed=123)
lhs_2D = lhs_engine.random(n=n)

# Generate evenly spaced grid
x = np.linspace(0, 1, n)
y = np.linspace(0, 1, n)
grid_x, grid_y = np.meshgrid(x, y)
grid_2D = np.vstack((grid_x.flatten(), grid_y.flatten())).T

# Plot the samples
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(lhs_2D[:, 0], lhs_2D[:, 1], c='red', marker='o')
plt.title('Latin Hypercube Sampling')
plt.xlabel('X1')
plt.ylabel('X2')

plt.subplot(1, 2, 2)
plt.scatter(grid_2D[:, 0], grid_2D[:, 1], c='blue', marker='o')
plt.title('Evenly Spaced Grid')
plt.xlabel('X1')
plt.ylabel('X2')

plt.show()



Latine Hypercube Sampling : add some randomness to grid points<br>
라틴 하이퍼큐브 샘플링 : 격자로 나눈 후 무작위성을 추가



In [ ]:
logger_shgo = RoughCostFunctionLogger()



In [ ]:
%%time
result_shgo = so.shgo(logger_shgo.cost, ((-6.0, 6.0),))
result_shgo



In [ ]:
logger_shgo.plot_cost_function_and_log()
plt.axvline(result_shgo.x, color="red");
plt.show()
plt.close()



### Dual annealing



Maintains an internal state `temperature`; if high, search for bigger regions ; vice versa. Another parameter `cooling_schedule` controls the `temperature` change.<br>
`temperature` 라는 내부 상태 변수를 두고 높으면 너른 범위를, 낮으면 좁은 범위를 탐색. `cooling_schedule` 이라는 다른 매개변수가 `temperature` 의 변화를 제어.



In [ ]:
logger_da = RoughCostFunctionLogger()



In [ ]:
%%time
result_da = so.dual_annealing(logger_da.cost, ((-6.0, 6.0),))
result_da



In [ ]:
logger_da.plot_cost_function_and_log()
plt.axvline(result_da.x, color="red");
plt.show()
plt.close()



### Comparison<br>비교



|   algorithm   | $x_{min}$ | # iter | # `f()` call | time |
|:-------------:|:---------:|:------:|:----------:|:----------:|
| Basin Hopping | -0.15629813 | 4 | 16 | 126 ms |
| Brute | -0.78145559 |   |   | 3.9 ms |
| Differential Evolution | -0.15629814 | 10 | 173 | 16.6 ms |
| SHGO | -0.15629813 | 2 | 16 | 1.95 ms |
| Dual annealing | -0.15629814 | 1000 | 2027 | 124 ms |



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

